In [72]:
import json
from natasha import NamesExtractor
import re
import numpy as np
import pickle
import requests
from pathlib import Path
import nltk
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords
import pymorphy2
from natasha.markup import show_markup_notebook as show_markup
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
path_df = '..\\02. Data creation\\df.pickle'

In [14]:
with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [15]:
df.head()

,topic,text
0,Какой поступок вы назвали бы самым трусливым?,С понятием трусости у меня идет прямая ассоциа...
1,Почему общество часто не ценит великих людей?,В истории нашей страны существует немало лично...
2,Какие цели важно ставить на жизненном пути?,"Как бы это банально не звучало, но мы живем дл..."
3,Что значит быть отзывчивым?,"Отзывчивость – это емкое понятие, вмещающее в ..."
4,Всегда ли хороша верность?,"Верность, доверие, любовь, уважение, поддержка..."


In [5]:
url_stopwords_ru = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.txt"


def get_text(url, encoding='utf-8', to_lower=True):
    url = str(url)
    if url.startswith('http'):
        r = requests.get(url)
        if not r.ok:
            r.raise_for_status()
        return r.text.lower() if to_lower else r.text
    elif os.path.exists(url):
        with open(url, encoding=encoding) as f:
            return f.read().lower() if to_lower else f.read()
    else:
        raise Exception('parameter [url] can be either URL or a filename')


def normalize_tokens(tokens):
    morph = pymorphy2.MorphAnalyzer()
    return [morph.parse(tok)[0].normal_form for tok in tokens]


def remove_stopwords(tokens, stopwords=None, min_length=4):
    if not stopwords:
        return tokens
    stopwords = set(stopwords)
    tokens = [tok
              for tok in tokens
              if tok not in stopwords and len(tok) >= min_length]
    return tokens


def tokenize_n_lemmatize(
    text, stopwords=None, normalize=True, 
    regexp=r'(?u)\b\w{3,}\b'):
    words = [w for sent in sent_tokenize(text)
             for w in regexp_tokenize(sent, regexp)]
    if normalize:
        words = normalize_tokens(words)
    if stopwords:
        words = remove_stopwords(words, stopwords)
    return words

stopwords_ru = get_text(url_stopwords_ru).splitlines()

In [6]:
def proc_df(df):
    nrows = len(df)
    #text_parsed_list = []
    topic_parsed_list = []
    for row in range(0, nrows):
        #text = df.loc[row]['text']
        topic = df.loc[row]['topic']
        #text = text.replace('\n', ' ')
        #text_parsed_list.append(' '.join(tokenize_n_lemmatize(text)))
        topic_parsed_list.append(' '.join(tokenize_n_lemmatize(topic)))
    #df['text_parsed'] = text_parsed_list
    df['topic_parsed'] = topic_parsed_list

In [44]:
proc_df(df)

In [45]:
df.head()

,topic,text,topic_parsed
0,Какой поступок вы назвали бы самым трусливым?,С понятием трусости у меня идет прямая ассоциа...,какой поступок назвать самый трусливый
1,Почему общество часто не ценит великих людей?,В истории нашей страны существует немало лично...,почему общество часто ценить великое человек
2,Какие цели важно ставить на жизненном пути?,"Как бы это банально не звучало, но мы живем дл...",какой цель важный ставить жизненный путь
3,Что значит быть отзывчивым?,"Отзывчивость – это емкое понятие, вмещающее в ...",что значит быть отзывчивый
4,Всегда ли хороша верность?,"Верность, доверие, любовь, уважение, поддержка...",всегда хороший верность


### Создадим Bag of Words для именованных сущностей

In [142]:
# Функция расчет схожести строк по коэффициенту Жаккара
def dist_jaccard(str1, str2):
    a = len(str1)
    b = len(str2)
    c = 0
    while str1 != '':
        s = str1[0]
        r = re.compile(s)
        c += min(len(r.findall(str1)), len(r.findall(str2)))
        str1 = r.sub('', str1)
        str2 = r.sub('', str2)
    return c / (a + b - c)

# Функция создания массива, который выделяет похожие строки в группы 
def make_groups(names):
    n = len(names)
    groups = np.array([-1] * n)
    gr = 1
    for i in range(n):
        j = i + 1
        while j != n:
            if dist_jaccard(names[i], names[j]) >= 0.6:
                if groups[i] == -1 and groups[j] == -1:
                    groups[i] = gr
                    groups[j] = gr
                    gr += 1
                elif groups[i] == -1:
                    groups[i] = groups[j]
                elif groups[j] == -1:
                    groups[j] = groups[i]
                elif groups[i] != groups[j]:
                    groups[np.where(groups == groups[j])] = groups[i]
            j += 1
    for i in range(n):
        if groups[i] == -1:
            groups[i] = gr
            gr += 1
    return groups

# Функция выделения именованных сущностей из текста и распределения их по группам
def extract_names(text):
    extractor = NamesExtractor()
    matches = extractor(text)
    result = np.array([])
    for match in matches:
        name = ''
        if match.fact.first != None:
                name += match.fact.first
        if match.fact.middle != None:
                name += ' ' + match.fact.middle
        if match.fact.last != None:
                name += ' ' + match.fact.last
        name = name.lower()
        result= np.append(result, name)
    return result, make_groups(result)

# Функция сравнения двух групп именовааных сущностей
def compare_groups(group1, group2):
    n = 0
    agg_dj = 0
    for el1 in group1:
        for el2 in group2:
            agg_dj += dist_jaccard(el1, el2)
            n += 1

    agg_dj /= n
    if agg_dj >=0.6:
        return True
    else:
        return False

    
def normalize_matrix_by_column(matrix):
    n = matrix.shape[0]
    m = matrix.shape[1]
    norm_matrix = np.zeros((n, m))
    for j in range(m):
        nu = np.mean(matrix[:,j])
        std = np.std(matrix[:,j])
        for i in range(n):
            if std != 0:
                norm_matrix[i,j] = float((matrix[i,j]-nu)/std) 
            else:
                norm_matrix[i,j] = 0.
    return norm_matrix

# Функция создания матрицы: строки - тексты, столбцы - именованные сущности (а точнее группы)
def make_named_enteties_matrix(df):
    groups_in_matrix = np.array([])
    text_group_pairs = []
    n = len(df)
    
    for i in range(n):
        names, groups = extract_names(df.loc[i]['text'])
        set_of_groups = set(groups)
        
        for group in set_of_groups:
            group_in_text = names[np.where(groups == group)]
            is_group_found = False
            m = len(groups_in_matrix)
            
            for j in range(m):
                if compare_groups(group_in_text, groups_in_matrix[j]):
                    groups_in_matrix[j] = groups_in_matrix[j].union(set(group_in_text))
                    text_group_pairs.append((i,j))
                    is_group_found = True
                    break
                    
            if not is_group_found:
                groups_in_matrix = np.append(groups_in_matrix, set(group_in_text))
                text_group_pairs.append((i,m))
                
    m = len(groups_in_matrix)
    matrix = np.zeros((n,m))
    for el in text_group_pairs:
        matrix[el] += 1
    return matrix, groups_in_matrix

In [159]:
named_entity_matrix, groups_in_matrix = make_named_enteties_matrix(df)
print(groups_in_matrix[:10])

[{'швабрин', ' швабрин'} {'пётр гринев', ' гринев'} {'евгений онегин'}
 {' онегин'} {'евгений'}
 {'иван николаевич понырёв', 'алексей иванович швабрин', 'николай иванович'}
 {'пётр'} {' пугачев'} {'владимир ленский'} {'данко'}]


### Составим матрицу Bag of Words для тем сочинений

In [160]:
vectorizer = CountVectorizer()

In [161]:
topic_matrix =  vectorizer.fit_transform(df['topic_parsed']).toarray()
print(topic_matrix.shape)

(43, 118)


In [162]:
words_in_topics = vectorizer.get_feature_names()

### Составим корреляционную матрицу между *topic_matrix* и *named_entity_matrix* 

In [146]:
from scipy.stats.stats import pearsonr

In [148]:
topic_matrix_norm = normalize_matrix_by_column(topic_matrix)
named_entity_matrix_norm = normalize_matrix_by_column(named_entity_matrix)

In [149]:
m = named_entity_matrix_norm.shape[1]
k = topic_matrix_norm.shape[1]
corr_matrix = np.zeros((m,k))
print(corr_matrix.shape)

(133, 118)


In [150]:
for i in range(m):
    for j in range(k):
        corr_matrix[i,j] = pearsonr(named_entity_matrix_norm[:,i], topic_matrix_norm[:,j])[0]

In [151]:
print(corr_matrix)

[[-0.04941662 -0.04941662 -0.07073279 ... -0.1025641  -0.04941662
  -0.04941662]
 [-0.03407991 -0.03407991 -0.04878049 ... -0.07073279 -0.03407991
  -0.03407991]
 [-0.06213698 -0.06213698 -0.08894014 ...  0.10209743 -0.06213698
  -0.06213698]
 ...
 [-0.02380952 -0.02380952 -0.03407991 ... -0.04941662 -0.02380952
  -0.02380952]
 [-0.02380952 -0.02380952 -0.03407991 ... -0.04941662 -0.02380952
  -0.02380952]
 [-0.02380952 -0.02380952 -0.03407991 ... -0.04941662 -0.02380952
  -0.02380952]]


In [158]:
z = 0
for i in range(m):
    for j in range(k):
        if corr_matrix[i,j] > 0.95:
            print(groups_in_matrix[i], words_in_topics[j])
            z += 1
            break
    if z == 10:
        break

{' пугачев'} назвать
{' чичиков'} цель
{'а и солженицын'} значит
{' желтковая'} верность
{' свидригайлов'} оправдывать
{' старцев'} опасный
{'вернер'} друг
{' ленский'} враг
{'и а бунин'} история
{'руфь'} жить


In [163]:
with open('named_entity_matrix.pickle', 'wb') as output:
    pickle.dump(named_entity_matrix, output)
    
with open('groups_in_matrix.pickle', 'wb') as output:
    pickle.dump(groups_in_matrix, output)

with open('vectorizer.pickle', 'wb') as output:
    pickle.dump(vectorizer, output)

with open('df_parsed.pickle', 'wb') as output:
    pickle.dump(df, output)